In [ ]:
import numpy as np
import pandas as pd
import glob
import os
import scipy.stats as ss
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from zipfile import ZipFile


  
# #specifying the zip file name
# for i in range(5):
#     file_name = f'Dag 2/Mg_data/Mg_1_maalning_dag2_ch000{i}.zip'
    
#     # opening the zip file in READ mode
#     with ZipFile(file_name, 'r') as ti:
#         # printing all the contents of the zip file
#         ti.printdir()
    
#         # extracting all the files
#         ti.extractall()

# a = 0.73971712
# ae = 9.9e-7
# b = 0.3785
# be = 1.4e-3

# data = []
# 
#     data.append(np.loadtxt(f'Mg_1_maalning_dag2_ch000{i}.txt')[1])

def getCounts(j, lc: int = 20, hc: int = 6000):
    
    counts = []
    for i in range(j):
        print('File:', str(i))
        data = np.loadtxt(f"Mg_1_maalning_dag2_ch000{i}.txt")
        counts.append(data[:, 1])
    
    (x, y) = np.unique(np.concatenate(counts), return_counts=True)
    lI = np.where(x >= lc)[0][0]
    hI = np.where(x >= hc)[0][0]
    x = x[lI:hI]
    y = y[lI:hI]
    
    a = 0.73971712
    ae = 9.9e-7
    b = 0.3785
    be = 1.4e-3
    x  = x * a + b
    
    a = 180.66341379895778
    b = 0.034598909761420625
    usikkerheder: [6.52763544e+00, 5.95806196e-03]    
    eff = a *1 / x + b
    y = y / eff


    return (x, y)

Line = [3445.279,3369.91,3122.908,2959.935,2657.547,2085.064,846.77]
line = [2598.438, 3369.81,2523.06,1037.833,2113.092,1810.726, 1238.27]


plt.vlines(Line, 0, 10000, colors='r', linestyles='dashed')
plt.vlines(line, 0, 10000, colors='b', linestyles='dashed')



Mg1 = getCounts(5)
# Mg2 = getCounts(1)
# Mg3 = getCounts(2)
# Mg4 = getCounts(3)
# Mg5 = getCounts(4)


# ener = np.concatenate([Mg1[0], Mg2[0] , Mg3[0] , Mg4[0] , Mg5[0]])
# Coun = np.concatenate([Mg1[1], Mg2[1] , Mg3[1] , Mg4[1] , Mg5[1]])

plt.plot(Mg1[0], Mg1[1], color="r", label="data")
plt.show()

# plt.vlines(Line, 0, 10000, colors='r', linestyles='dashed')
# plt.vlines(line, 0, 10000, colors='b', linestyles='dashed')

plt.plot(Mg1[0], Mg1[1])
plt.title('Mg')

plt.show()




def gaussFit(x, mu, sig, a, b, c):
    lny = np.log(a) - ((x-mu)**2)/(2*sig**2)
    return np.exp(lny) - (b*x+c)



In [ ]:

def getChannel(name: str, data:tuple , lower_limit: int, upper_limit: int, x_1:int, x_2:int, guess: [int, int, int], guess2 = [0,0]):
    x = data[0][lower_limit:upper_limit]
    y = data[1][lower_limit:upper_limit]


    print('x_min',x[0],'x_max',x[-1], 'y_min',y[0],'y_max', y[-1])
    plt.plot(x, y, color="r", label="data")
    yler = np.sqrt(y)
    pinit = guess + guess2
    xhelp = np.linspace(x_1, x_2, 500)
    print('heplmin',xhelp[0],'helpmax',xhelp[-1])
    popt, pcov = curve_fit(gaussFit, x, y, p0=pinit, sigma=yler, absolute_sigma=True)
    print(name)
    print('mu :', popt[0])
    print('sigma :', popt[1])
    print('scaling', popt[2])
    print('background', popt[3], popt[4])
    perr = np.sqrt(np.diag(pcov))
    print('usikkerheder:', perr)
    chmin = np.sum(((y - gaussFit(x, *popt)) / yler) ** 2)
    print('chi2:', chmin, ' ---> p:', ss.chi2.cdf(chmin, 4))

    
    plt.plot(xhelp, gaussFit(xhelp, *popt), 'k-.', label="gaussfit")
    plt.legend()

    plt.title(name)
    plt.grid()
    plt.show()


    return [popt, perr, name]


In [ ]:
def t(x):
    return int(x/ a - b)

def effect(x):
    a = 180.66341379895778
    b = 0.034598909761420625
    usikkerheder: [6.52763544e+00, 5.95806196e-03]    
    return a *1 / x +b


In [ ]:
chs = []

chs += [getChannel("Mg E=846.77", Mg1,  1000, 1500, 750, 1000, [850, 10, 200])]
chs += [getChannel("Mg E=2085.0", Mg1, 2780, 2802, 2075,2088, [2085, 10, 100])]    
chs += [ getChannel("Mg E=1238.27", Mg1, 1500, 1800,1100,1300, [1230, 10, 200])]
chs += [ getChannel("Mg E=2657.54", Mg1, 3000, 4000, 2500,2800, [2650, 5, 200])]
chs += [ getChannel("Mg E=1810.72", Mg1, 2300, 2600, 1750,1900,[1810, 25, 200])]
chs += [ getChannel("Mg E=2959.03", Mg1, 3800, 4100, 2900,3000, [2950, 10, 200])] 
chs += [getChannel("Mg E=3122.908", Mg1, 4100, 4270, 3000, 3200, [3122, 0, 200])]
chs += [ getChannel("Mg E=2113.09", Mg1, 2720, 3020, 2000,2200, [2120, 10, 200])]
chs += [ getChannel("Mg E=1037.833", Mg1, 1340, 1440,1020,1050, [1037, 0.5, 5000])]
chs += [ getChannel("Mg E=3369.9", Mg1, 4100, 5400,3200, 3500, [3360, 10, 200])]
chs += [ getChannel("Mg E=2523.06", Mg1, 3200, 3600,2400,2600, [2520, 20, 500])]
chs += [ getChannel("Mg E=2598.43", Mg1, 3450, 3498,2580,2600, [2598, 10, 20])]

In [ ]:
ch= np.array(chs)
ch

In [ ]:
#A = H * Sigma \sqrt(2pi)

def Areal(t):
    H = t[2]
    Sigma = t[1]
    A = H * Sigma * (2*np.pi)**0.5
    return A

Areals = [Areal(i) for i in ch[:,0]]
Areals

In [ ]:

na = {}
for i in range(len(Areals)):
    na.update({chs[i][2][5:]: Areals[i]})

print(na)
E7 = na['2598.43']
E6 = na['3369.9'] + na['2523.06']
E5 = na['1037.833']
E4 = na['2959.03'] + na['2113.09']
E3 = na['2657.54'] + na['1810.72']
E2 = na['1238.27'] - na['1037.833']
E1 = na['846.77'] - na['1238.27'] - na['1810.72'] - na['2113.09'] - na['2523.06'] - na['2598.43']

E = E7, E6, E5, E4, E3, E2, E1

print(E)

In [ ]:
print(E7/sum(E), E6/sum(E), E5/sum(E), E4/sum(E), E3/sum(E), E2/sum(E), E1/sum(E))